# Part A: Preprocessing

### **Authors:** oscardong4@gmail.com, thomas.oneil@sydney.edu.au & heeva.baharlou@sydney.edu.com (Dec 2024) - script adapted from [here](https://github.com/BodenmillerGroup/ImcSegmentationPipeline/blob/main/scripts/imc_preprocessing.ipynb)

To fill in extra info


## Order of the analysis
1. MCD extraction
2. Cellpose prep
3. Cellpose model training
4. Cellpose batch segmentation
5. Feature Extraction

# Setting variables

text

In [ ]:
# Set this to your 'analysis' folder
analysis = ""

# Set this to your 'raw' folder
raw = ""

# MCD extraction

**MCD extraction**  
<span style="color:grey; opacity: 0.5">Cellpose prep</span>  
<span style="color:grey; opacity: 0.5">Cellpose model training</span>  
<span style="color:grey; opacity: 0.5">Cellpose batch segmentation</span>    
<span style="color:grey; opacity: 0.5">Feature Extraction</span>    

In [69]:
denoise = True

In [71]:
# Import libraries
from pathlib import Path
from tempfile import TemporaryDirectory
import pandas as pd
import tifffile as tiff
import numpy as np
import imcsegpipe
from imcsegpipe.utils import sort_channels_by_mass

# Logical variable for denoising
denoise = True

# Working directory storing all outputs
work_dir = Path(analysis)
work_dir.mkdir(exist_ok=True)

# Set and create output directories
acquisitions_dir = work_dir / "1a_extracted_mcd"
segment_dir = work_dir / "1b_for_segmentation"
output_dir = work_dir / "1c_full_images"
denoise_dir = work_dir / "1d_denoise" if denoise else None
acquisitions_dir.mkdir(exist_ok=True)
segment_dir.mkdir(exist_ok=True)
output_dir.mkdir(exist_ok=True)
if denoise:
    denoise_dir.mkdir(exist_ok=True)

# Raw directory with raw data files
raw = Path(raw)

# Step 1: Extract .mcd files
temp_dirs = []
try:
    for raw_dir in [raw]:
        zip_files = list(raw_dir.rglob("**/*.zip"))
        if len(zip_files) > 0:
            temp_dir = TemporaryDirectory()
            temp_dirs.append(temp_dir)
            for zip_file in sorted(zip_files):
                imcsegpipe.extract_zip_file(zip_file, temp_dir.name)
    for raw_dir in [raw] + [Path(temp_dir.name) for temp_dir in temp_dirs]:
        mcd_files = list(raw_dir.rglob("*.mcd"))
        mcd_files = [i for i in mcd_files if not i.stem.startswith('.')]
        if len(mcd_files) > 0:
            txt_files = list(raw_dir.rglob("*.txt"))
            txt_files = [i for i in txt_files if not i.stem.startswith('.')]
            matched_txt_files = imcsegpipe.match_txt_files(mcd_files, txt_files)
            for mcd_file in mcd_files:
                imcsegpipe.extract_mcd_file(
                    mcd_file,
                    acquisitions_dir / mcd_file.stem,
                    txt_files=matched_txt_files[mcd_file]
                )
finally:
    for temp_dir in temp_dirs:
        temp_dir.cleanup()
    del temp_dirs

# Read the panel.csv
panel = pd.read_csv(raw / "panel.csv")

# Step 2: Generate image stacks (_full and _segment) for 1b and 1c
for acquisition_dir in acquisitions_dir.glob("[!.]*"):
    if acquisition_dir.is_dir():
        imcsegpipe.create_analysis_stacks(
            acquisition_dir=acquisition_dir,
            analysis_dir=output_dir,
            analysis_channels=sort_channels_by_mass(
                panel.loc[panel["Full"] == 1, "Metal Tag"].tolist()
            ),
            suffix="_full",
            hpf=50.0
        )
        imcsegpipe.create_analysis_stacks(
            acquisition_dir=acquisition_dir,
            analysis_dir=segment_dir,
            analysis_channels=sort_channels_by_mass(
                panel.loc[panel["Segment"] == 1, "Metal Tag"].tolist()
            ),
            suffix="_segment",
            hpf=50.0
        )

# Step 3: Process TIFFs for denoising
if denoise:
    for sample_dir in acquisitions_dir.glob("[!.]*"):
        if sample_dir.is_dir():
            for roi_tiff_path in sample_dir.glob("*.tiff"):
                roi_name = roi_tiff_path.stem
                roi_subdir = denoise_dir / roi_name
                roi_subdir.mkdir(parents=True, exist_ok=True)

                # Load the stack using tifffile
                with tiff.TiffFile(roi_tiff_path) as tif:
                    stack = tif.asarray()  # Load the entire TIFF stack as a NumPy array

                # Filter and unstack based on panel.csv
                for idx, row in panel[panel["Full"] == 1].iterrows():
                    metal_tag = row["Metal Tag"]
                    target = row["Target"]
                    output_name = f"{metal_tag}-{target}_{metal_tag}.tiff"
                    output_path = roi_subdir / output_name

                    # Extract the specific slice from the stack
                    slice_image = stack[idx, :, :]  # Adjust indexing based on stack structure

                    # Save the slice as a TIFF
                    tiff.imwrite(output_path, slice_image.astype(np.uint16))  # Save as 16-bit TIFF

print("Done!")


/Users/thomasoneil/anaconda3/envs/imcsegpipe/lib/python3.9/site-packages/readimc/mcd_parser.py:131: UserWarning: Slide 0 corrupted: overlapping memory blocks for acquisitions 1 and 2
  warn(
ERROR:root:Error reading acquisition 5 from file 240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J.mcd: MCD file '240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J.mcd' corrupted: inconsistent acquisition image data size
ERROR:root:Error reading acquisition 6 from file 240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J.mcd: MCD file '240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J.mcd' corrupted: inconsistent acquisition image data size
/Users/thomasoneil/anaconda3/envs/imcsegpipe/lib/python3.9/site-packages/readimc/mcd_parser.py:131: UserWarning: Slide 0 corrupted: overlapping memory blocks for acquisitions 1 and 7
  warn(
/Users/thomasoneil/anaconda3/envs/imcsegpipe/lib/python3.9/site-packages/readimc/mcd_parser.py:131: UserWarning: Slide 0 corrupted: overlapping memory blocks for acquisitio

Done!


Tested on Siddheshs data. Samples 1:3. Took 38 seconds on my Mac. 5GB output from 1.47GB input. Denoise outputs files labelled in accordance with IMC_Denoise pipeline (metaltag-marker_metaltag)

# Cellpose preparation

<strike>MCD extraction</strike>  
**Cellpose prep**  
<span style="color:grey; opacity: 0.5">Cellpose model training</span>  
<span style="color:grey; opacity: 0.5">Cellpose batch segmentation</span>    
<span style="color:grey; opacity: 0.5">Feature Extraction</span>    

Set your variables before running. Identify the `DNA` channel and the `square size` (in pixels) you want to use for cellpose training

In [59]:
dna = "DNA"
square_size = 200

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from skimage import io, exposure, img_as_uint

# Define directories
dir_images = os.path.join(analysis, "1b_for_segmentation")
im_output = os.path.join(analysis, "2a_cellpose_full")
crop_output = os.path.join(analysis, "2b_cropped_images")
panel_file = os.path.join(raw, "panel.csv")

# Create output directories
os.makedirs(im_output, exist_ok=True)
os.makedirs(crop_output, exist_ok=True)

# load image list
image_list = [f for f in os.listdir(dir_images) if f.endswith(('.tiff', '.tif'))]

# read panel
panel = pd.read_csv(panel_file)
segmentation_targets = panel.loc[panel['Segment'] == 1, 'Target'].tolist()
print("Segmentation Targets:", segmentation_targets)

# get indices of dna channel
dna_index = [i for i, target in enumerate(segmentation_targets) if target == dna]

# crop and compress each image
for image_file in image_list:
    image_path = os.path.join(dir_images, image_file)
    image = io.imread(image_path)
    im_title = os.path.splitext(image_file)[0]
    
    # normalise
    normalized_stack = []
    for i in range(image.shape[0]): 
        channel = image[i, :, :]
        normalized = exposure.rescale_intensity(channel, in_range='image', out_range=(0, 1))
        normalized_stack.append(img_as_uint(normalized))
    normalized_stack = np.stack(normalized_stack)
    
    # get dna channel
    if dna_index:
        # keep only the first instance of dna
        dna_channel = normalized_stack[dna_index[0]]
        
        # remove dna from segmentation stack
        for idx in sorted(dna_index, reverse=True):
            normalized_stack = np.delete(normalized_stack, idx, axis=0)
    else: #error message if dna not found
        raise ValueError("DNA channel not found in segmentation targets.")
    
    # create mask for surface segmentation
    surface_mask = np.mean(normalized_stack, axis=0).astype(np.uint16)
    
    # create empty channel - for cellpose colour scheme to avoid red/green and combine in order empty > segment > dna
    empty_channel = np.zeros_like(dna_channel, dtype=np.uint16)
    empty -> surface mask -> DNA
    composite_stack = np.stack([empty_channel, surface_mask, dna_channel])
    
    # save
    im_output_path = os.path.join(im_output, f"{im_title}_CpSeg.tiff")
    io.imsave(im_output_path, composite_stack)
    
    # get crop dimensions
    height, width = composite_stack.shape[1:3]
    if width < square_size or height < square_size:
        # if image is smaller than crop size, save image itself as the crop
        crop_output_path = os.path.join(crop_output, f"{im_title}_CpCrop.tiff")
        io.imsave(crop_output_path, composite_stack)
        print(f"Image {im_title} is smaller than the cropping size. Saved without cropping.")
        continue

    # create the crop and save
    workable_x = width - square_size
    workable_y = height - square_size
    rand_x = random.randint(0, workable_x)
    rand_y = random.randint(0, workable_y)
    cropped = composite_stack[:, rand_y:rand_y + square_size, rand_x:rand_x + square_size]
    crop_output_path = os.path.join(crop_output, f"{im_title}_CpCrop.tiff")
    io.imsave(crop_output_path, cropped)
print("Done!")

Segmentation Targets: ['aSMA', 'CLA', 'anti-Cy3', 'CD68', 'CD163', 'CD183', 'FXIIIa', 'Ki67', 'anti-Biotin', 'anti-Cy5', 'CD3', 'CD206', 'HLA-DR', 'DNA', 'DNA']
Image 300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a2_ac_segment is smaller than the cropping size. Saved without cropping.
Processed and cropped 240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J_s0_a2_ac_segment.
Image 240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J_s0_a5_ac_segment is smaller than the cropping size. Saved without cropping.
Processed and cropped 300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a5_ac_segment.


/var/folders/h5/tq0dp1p95rsfkwyjmhp757hr0000gn/T/ipykernel_9297/3693344009.py:66: UserWarning: /Users/thomasoneil/Library/CloudStorage/OneDrive-TheUniversityofSydney(Staff)/DROPBOX/siddhesh/analysis/2a_cellpose_full/300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a5_ac_segment_CpSeg.tiff is a low contrast image
  io.imsave(im_output_path, composite_stack)
/var/folders/h5/tq0dp1p95rsfkwyjmhp757hr0000gn/T/ipykernel_9297/3693344009.py:85: UserWarning: /Users/thomasoneil/Library/CloudStorage/OneDrive-TheUniversityofSydney(Staff)/DROPBOX/siddhesh/analysis/2b_cropped_images/300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a5_ac_segment_CpCrop.tiff is a low contrast image
  io.imsave(crop_output_path, cropped)
/var/folders/h5/tq0dp1p95rsfkwyjmhp757hr0000gn/T/ipykernel_9297/3693344009.py:85: UserWarning: /Users/thomasoneil/Library/CloudStorage/OneDrive-TheUniversityofSydney(Staff)/DROPBOX/siddhesh/analysis/2b_cropped_images/240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J_s0_a4_ac

Processed and cropped 240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J_s0_a4_ac_segment.
Processed and cropped 240824_Cohort_Study_SL3_H13-1010A_H13-1010B_H13-482C_s0_a1_ac_segment.
Image 300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a4_ac_segment is smaller than the cropping size. Saved without cropping.
Image 300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a3_ac_segment is smaller than the cropping size. Saved without cropping.
Processed and cropped 240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J_s0_a3_ac_segment.
Image 240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J_s0_a6_ac_segment is smaller than the cropping size. Saved without cropping.
Processed and cropped 300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a9_ac_segment.


/var/folders/h5/tq0dp1p95rsfkwyjmhp757hr0000gn/T/ipykernel_9297/3693344009.py:85: UserWarning: /Users/thomasoneil/Library/CloudStorage/OneDrive-TheUniversityofSydney(Staff)/DROPBOX/siddhesh/analysis/2b_cropped_images/300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a9_ac_segment_CpCrop.tiff is a low contrast image
  io.imsave(crop_output_path, cropped)


Processed and cropped 300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a6_ac_segment.
Processed and cropped 240824_Cohort_Study_SL3_H13-1010A_H13-1010B_H13-482C_s0_a3_ac_segment.
Processed and cropped 240824_Cohort_Study_SL3_H13-1010A_H13-1010B_H13-482C_s0_a4_ac_segment.
Image 300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a1_ac_segment is smaller than the cropping size. Saved without cropping.
Processed and cropped 240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J_s0_a1_ac_segment.


/var/folders/h5/tq0dp1p95rsfkwyjmhp757hr0000gn/T/ipykernel_9297/3693344009.py:85: UserWarning: /Users/thomasoneil/Library/CloudStorage/OneDrive-TheUniversityofSydney(Staff)/DROPBOX/siddhesh/analysis/2b_cropped_images/240824_Cohort_Study_SL2_H13-908B_H13-908A_H17-334J_s0_a1_ac_segment_CpCrop.tiff is a low contrast image
  io.imsave(crop_output_path, cropped)


Processed and cropped 300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a8_ac_segment.
Processed and cropped 300824_Cohort_Study_SL4_H16-794A_H16-794B_H17-350A_s0_a7_ac_segment.
Processed and cropped 240824_Cohort_Study_SL3_H13-1010A_H13-1010B_H13-482C_s0_a2_ac_segment.


/var/folders/h5/tq0dp1p95rsfkwyjmhp757hr0000gn/T/ipykernel_9297/3693344009.py:85: UserWarning: /Users/thomasoneil/Library/CloudStorage/OneDrive-TheUniversityofSydney(Staff)/DROPBOX/siddhesh/analysis/2b_cropped_images/240824_Cohort_Study_SL3_H13-1010A_H13-1010B_H13-482C_s0_a2_ac_segment_CpCrop.tiff is a low contrast image
  io.imsave(crop_output_path, cropped)
